In [ ]:
import numpy as np
from scipy.io import loadmat
from scipy.optimize import minimize
import pickle
from sklearn.svm import SVC


def preprocess():
    """ 
     Input:
     Although this function doesn't have any input, you are required to load
     the MNIST data set from file 'mnist_all.mat'.

     Output:
     train_data: matrix of training set. Each row of train_data contains 
       feature vector of a image
     train_label: vector of label corresponding to each image in the training
       set
     validation_data: matrix of training set. Each row of validation_data 
       contains feature vector of a image
     validation_label: vector of label corresponding to each image in the 
       training set
     test_data: matrix of training set. Each row of test_data contains 
       feature vector of a image
     test_label: vector of label corresponding to each image in the testing
       set
    """

    mat = loadmat('mnist_all.mat')  # loads the MAT object as a Dictionary

    n_feature = mat.get("train1").shape[1]
    n_sample = 0
    for i in range(10):
        n_sample = n_sample + mat.get("train" + str(i)).shape[0]
    n_validation = 1000
    n_train = n_sample - 10 * n_validation

    # Construct validation data
    validation_data = np.zeros((10 * n_validation, n_feature))
    for i in range(10):
        validation_data[i * n_validation:(i + 1) * n_validation, :] = mat.get("train" + str(i))[0:n_validation, :]

    # Construct validation label
    validation_label = np.ones((10 * n_validation, 1))
    for i in range(10):
        validation_label[i * n_validation:(i + 1) * n_validation, :] = i * np.ones((n_validation, 1))

    # Construct training data and label
    train_data = np.zeros((n_train, n_feature))
    train_label = np.zeros((n_train, 1))
    temp = 0
    for i in range(10):
        size_i = mat.get("train" + str(i)).shape[0]
        train_data[temp:temp + size_i - n_validation, :] = mat.get("train" + str(i))[n_validation:size_i, :]
        train_label[temp:temp + size_i - n_validation, :] = i * np.ones((size_i - n_validation, 1))
        temp = temp + size_i - n_validation

    # Construct test data and label
    n_test = 0
    for i in range(10):
        n_test = n_test + mat.get("test" + str(i)).shape[0]
    test_data = np.zeros((n_test, n_feature))
    test_label = np.zeros((n_test, 1))
    temp = 0
    for i in range(10):
        size_i = mat.get("test" + str(i)).shape[0]
        test_data[temp:temp + size_i, :] = mat.get("test" + str(i))
        test_label[temp:temp + size_i, :] = i * np.ones((size_i, 1))
        temp = temp + size_i

    # Delete features which don't provide any useful information for classifiers
    sigma = np.std(train_data, axis=0)
    index = np.array([])
    for i in range(n_feature):
        if (sigma[i] > 0.001):
            index = np.append(index, [i])
    train_data = train_data[:, index.astype(int)]
    validation_data = validation_data[:, index.astype(int)]
    test_data = test_data[:, index.astype(int)]

    # Scale data to 0 and 1
    train_data /= 255.0
    validation_data /= 255.0
    test_data /= 255.0

    return train_data, train_label, validation_data, validation_label, test_data, test_label


def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))


def blrObjFunction(initialWeights, *args):
    """
    blrObjFunction computes 2-class Logistic Regression error function and
    its gradient.

    Input:
        initialWeights: the weight vector (w_k) of size (D + 1) x 1 
        train_data: the data matrix of size N x D
        labeli: the label vector (y_k) of size N x 1 where each entry can be either 0 or 1 representing the label of corresponding feature vector

    Output: 
        error: the scalar value of error function of 2-class logistic regression
        error_grad: the vector of size (D+1) x 1 representing the gradient of
                    error function
    """
    train_data, labeli = args
    
    n_data = train_data.shape[0]
    n_features = train_data.shape[1]
    error = 0
    error_grad = np.zeros((n_features + 1, 1))
    w = initialWeights

    ##################
    # YOUR CODE HERE #
    ##################
    # HINT: Do not forget to add the bias term to your input data
    w = w.reshape((n_features+1,1))
    
    x = np.hstack((np.ones((n_data,1)),train_data))
    Z = sigmoid(np.dot(x,w))

    error = labeli * np.log(Z) + (1.0 - labeli) * np.log(1.0 - Z)
    error = -np.sum(error)

    error_grad = (Z - labeli) * x
    error_grad = np.sum(error_grad, axis=0)

    return error, error_grad


def blrPredict(W, data):
    """
     blrObjFunction predicts the label of data given the data and parameter W 
     of Logistic Regression
     
     Input:
         W: the matrix of weight of size (D + 1) x 10. Each column is the weight 
         vector of a Logistic Regression classifier.
         X: the data matrix of size N x D
         
     Output: 
         label: vector of size N x 1 representing the predicted label of 
         corresponding feature vector given in data matrix

    """ 
    label = np.zeros((data.shape[0], 1))
    ##################
    # YOUR CODE HERE #
    ##################
    # HINT: Do not forget to add the bias term to your input data
    
    
    
    n_data = data.shape[0]
    data = np.hstack([np.ones(n_data).reshape(n_data,1),data]) 

    label = sigmoid(np.dot(data,W)) 
    
    label = np.argmax(label,axis=1)
    label = label.reshape(n_data,1)

    return label


"""
Script for Logistic Regression
"""
train_data, train_label, validation_data, validation_label, test_data, test_label = preprocess()

# number of classes
n_class = 10

# number of training samples
n_train = train_data.shape[0]

# number of features
n_feature = train_data.shape[1]

Y = np.zeros((n_train, n_class))
for i in range(n_class):
    Y[:, i] = (train_label == i).astype(int).ravel()


# Logistic Regression with Gradient Descent
W = np.zeros((n_feature + 1, n_class))
initialWeights = np.zeros((n_feature + 1, 1))
opts = {'maxiter': 100}
for i in range(n_class):
    labeli = Y[:, i].reshape(n_train, 1)
    args = (train_data, labeli)
    nn_params = minimize(blrObjFunction, initialWeights, jac=True, args=args, method='CG', options=opts)
    W[:, i] = nn_params.x.reshape((n_feature + 1,))

# Find the accuracy on Training Dataset
predicted_label = blrPredict(W, train_data)
print('\n Training set Accuracy:' + str(100 * np.mean((predicted_label == train_label).astype(float))) + '%')

# Find the accuracy on Validation Dataset
predicted_label = blrPredict(W, validation_data)
print('\n Validation set Accuracy:' + str(100 * np.mean((predicted_label == validation_label).astype(float))) + '%')

# Find the accuracy on Testing Dataset
predicted_label = blrPredict(W, test_data)
print('\n Testing set Accuracy:' + str(100 * np.mean((predicted_label == test_label).astype(float))) + '%')



"""
Script for Support Vector Machine
"""

print('\n\n--------------SVM-------------------\n\n')
##################
# YOUR CODE HERE #
##################

f1 = open('params.pickle', 'wb') 
pickle.dump(W, f1) 
f1.close()



# Linear kernel
print('SVM with linear kernel')
classifier = SVC(kernel='linear')
classifier.fit(train_data, train_label.flatten())
print('\n Training set Accuracy:' + str(100*classifier.score(train_data, train_label)) + '%')
print('\n Validation set Accuracy:' + str(100*classifier.score(validation_data, validation_label)) + '%')
print('\n Testing set Accuracy:' + str(100*classifier.score(test_data, test_label)) + '%')


# Radial basis function with gamma = 1
print('\n\n SVM with radial basis function, gamma = 1')
classifier = SVC(kernel='rbf', gamma=1.0)
classifier.fit(train_data, train_label.flatten())
print('\n Training set Accuracy:' + str(100*classifier.score(train_data, train_label)) + '%')
print('\n Validation set Accuracy:' + str(100*classifier.score(validation_data, validation_label)) + '%')
print('\n Testing set Accuracy:' + str(100*classifier.score(test_data, test_label)) + '%')


# Radial basis function with gamma = 0
print('\n\n SVM with radial basis function, gamma = 0')
classifier = SVC(kernel='rbf')
classifier.fit(train_data, train_label.flatten())
print('\n Training set Accuracy:' + str(100*classifier.score(train_data, train_label)) + '%')
print('\n Validation set Accuracy:' + str(100*classifier.score(validation_data, validation_label)) + '%')
print('\n Testing set Accuracy:' + str(100*classifier.score(test_data, test_label)) + '%')


print('\n\n RBF, gamma = default, varying C values 1,10,20,...100 SVM \n')
cvalues = [1,10,20,30,40,50,60,70,80,90,100]
for i in cvalues:
    
    classifier = SVC(C=i,kernel='rbf')
    classifier.fit(train_data,train_label)
    predicted_values = classifier.predict(train_data);
    print('C =' + str(i)+'\n')
    print('\n Training set Accuracy:' + str(100*np.mean((predicted_values == train_label).astype(float))) + '%')
    predicted_values = classifier.predict(validation_data);
    print('\n Validation set Accuracy:' + str(100*np.mean((predicted_values == validation_label).astype(float))) + '%')
    predicted_values = classifier.predict(test_data);
    print('\n Testing set Accuracy:' + str(100*np.mean((predicted_values == test_label).astype(float))) + '%')



def mlrObjFunction(params, *args):
    """
    mlrObjFunction computes multi-class Logistic Regression error function and
    its gradient.

    Input:
        initialWeights: the weight vector of size (D + 1) x 1
        train_data: the data matrix of size N x D
        labeli: the label vector of size N x 1 where each entry can be either 0 or 1
                representing the label of corresponding feature vector

    Output:
        error: the scalar value of error function of multi-class logistic regression
        error_grad: the vector of size (D+1) x 10 representing the gradient of
                    error function
    """
    global train_data
    n_data = train_data.shape[0]
    n_feature = train_data.shape[1]
    error = 0
    error_grad = np.zeros((n_feature + 1, n_class))

    ##################
    # YOUR CODE HERE #
    ##################
    # HINT: Do not forget to add the bias term to your input data
    
    #Create bias term and add to data
    ones_bias = np.ones((n_data,1))

    train_data = np.concatenate((ones_bias,train_data),axis=1)

    #Create weight by reshaping
    W = params.reshape(n_feature + 1,n_class)

    #creating theta matrix
    theta = np.zeros((n_data, n_class))

    # For multi-class Logistic Regression, the posterior probabilities are given by a softmax transformation of
    # linear functions of the feature variables Equation(5)
    dot_product_W_X = np.dot(train_data, W)

    theta_matrix_sum = np.sum(np.exp(dot_product_W_X), axis=1).reshape(n_data, 1)

    theta_matrix = (np.exp(dot_product_W_X) / theta_matrix_sum)

    #log of theta matrix
    log_theta_matrix = np.log(theta_matrix)

    # The likelihood function with the negative logarithm Equation(7)
    error = (-1) * np.sum(np.sum(labeli * log_theta_matrix))

    # We now take the gradient of the error function with respect to one of the parameter vectors
    # w(k).  Making use of the result for the derivatives of the softmax function, we obtain: Equation(8)
    subtract_theta_y = theta_matrix - labeli

    error_grad = (np.dot(train_data.T, subtract_theta_y))

    return error, error_grad

def mlrPredict(W, data):
    """
     mlrObjFunction predicts the label of data given the data and parameter W
     of Logistic Regression

     Input:
         W: the matrix of weight of size (D + 1) x 10. Each column is the weight
         vector of a Logistic Regression classifier.
         X: the data matrix of size N x D

     Output:
         label: vector of size N x 1 representing the predicted label of
         corresponding feature vector given in data matrix

    """
    label = np.zeros((data.shape[0], 1))

    ##################
    # YOUR CODE HERE #
    ##################
    # HINT: Do not forget to add the bias term to your input data
    
    #Create bias term and add to data
    ones_bias = np.ones((data.shape[0],1))
    train_data = np.concatenate((ones_bias,data),axis=1)

    # For multi-class Logistic Regression, the posterior probabilities are given by a softmax transformation of
    # linear functions of the feature variables Equation(5)
    dot_product_W_X = np.dot(train_data, W)
    theta_matrix_sum = np.sum(np.exp(dot_product_W_X))

    posterior = np.exp(dot_product_W_X) / theta_matrix_sum

    for i in range(posterior.shape[0]):
        label[i] = np.argmax(posterior[i])
    label = label.reshape(label.shape[0], 1)

    return label

"""
#Script for Extra Credit Part
# FOR EXTRA CREDIT ONLY
W_b = np.zeros((n_feature + 1, n_class))
initialWeights_b = np.zeros((n_feature + 1, n_class))
opts_b = {'maxiter': 100}

args_b = (train_data, Y)
nn_params = minimize(mlrObjFunction, initialWeights_b, jac=True, args=args_b, method='CG', options=opts_b)
W_b = nn_params.x.reshape((n_feature + 1, n_class))

# Find the accuracy on Training Dataset
predicted_label_b = mlrPredict(W_b, train_data)
print('\n Training set Accuracy:' + str(100 * np.mean((predicted_label_b == train_label).astype(float))) + '%')

# Find the accuracy on Validation Dataset
predicted_label_b = mlrPredict(W_b, validation_data)
print('\n Validation set Accuracy:' + str(100 * np.mean((predicted_label_b == validation_label).astype(float))) + '%')

# Find the accuracy on Testing Dataset
predicted_label_b = mlrPredict(W_b, test_data)
print('\n Testing set Accuracy:' + str(100 * np.mean((predicted_label_b == test_label).astype(float))) + '%')
"""

C:\Users\yaniv\Anaconda2\lib\site-packages\ipykernel_launcher.py:125: RuntimeWarning: divide by zero encountered in log
C:\Users\yaniv\Anaconda2\lib\site-packages\ipykernel_launcher.py:125: RuntimeWarning: invalid value encountered in multiply
C:\Users\yaniv\Anaconda2\lib\site-packages\ipykernel_launcher.py:90: RuntimeWarning: overflow encountered in exp



 Training set Accuracy:84.93%

 Validation set Accuracy:83.69%

 Testing set Accuracy:84.1%


--------------SVM-------------------


SVM with linear kernel

 Training set Accuracy:97.286%
